In [ ]:
# Import modules
import os
import pandas as pd
import glob

from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

# from wordcloud import WordCloud
# from wordcloud import ImageColorGenerator
# from wordcloud import STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

data_path = './data/'

df_list = []

    
for data_file in glob.glob(data_path + '*.csv'):
    batch_df = pd.read_csv(data_file)
    df_list.append(batch_df)
    
    
data_df = pd.concat(df_list)
data_df = data_df.astype(str)
data_df

# Filtering out prompt keywords leaked into the generated data

In [ ]:
data_df = data_df.fillna('NaN')
data_df = data_df[~(data_df["corrected_reponse"].str.contains("Nothing"))]

topic = data_df[~(data_df["input"].str.contains("topic:"))]
Topic = topic[~(topic["input"].str.contains("Topic:"))]
question_type = Topic[~(Topic["input"].str.contains("question_type:"))]
Question_type = question_type[~(question_type["input"].str.contains("Question_type:"))]
type_ = Question_type[~(Question_type["input"].str.contains("type:"))]
Type = type_[~(type_["input"].str.contains("Type:"))]
question = Type[~(Type["input"].str.contains("question:"))]
Question = question[~(question["input"].str.contains("Question:"))]


input_ = Question[~(Question["initial_response"].str.contains("input:"))]
Input = input_[~(input_["initial_response"].str.contains("Input:"))]
response = Input[~(Input["initial_response"].str.contains("response:"))]
Response = response[~(response["initial_response"].str.contains("Response:"))]
corrector = Response[~(Response["initial_response"].str.contains("corrector:"))]
Corrector = corrector[~(corrector["initial_response"].str.contains("Corrector:"))]


input__ = Corrector[~(Corrector["corrected_reponse"].str.contains("input:"))]
Input_ = input__[~(input__["corrected_reponse"].str.contains("Input:"))]
response_ = Input_[~(Input_["corrected_reponse"].str.contains("response:"))]
Response_ = response_[~(response_["corrected_reponse"].str.contains("Response:"))]
corrector_ = Response_[~(Response_["corrected_reponse"].str.contains("corrector:"))]
Corrector_ = corrector_[~(corrector_["corrected_reponse"].str.contains("Corrector:"))]


data_df = Corrector_.copy()
data_df


# Filtering out responses that contain '<'

In [ ]:
# Filtering out responses that contain '<'
tags_df = data_df[data_df["initial_response"].str.contains("<")]
display(tags_df)
# tags_df.to_csv('./data/final/html_tags.csv', index = False)

data_df_no_tags = data_df[~(data_df["initial_response"].str.contains("<"))]
# data_df_no_tags = data_df
data_df_no_tags

# Filtering out 'none' responses 

In [ ]:
# Filtering out 'none' responses 
none_df1 = data_df_no_tags[data_df_no_tags["initial_response"]=="none"]
none_df2 = data_df_no_tags[data_df_no_tags["initial_response"]=="None"]
none_df = pd.concat([none_df1, none_df2])
display(none_df)
# none_df.to_csv('./data_care/combined/none.csv', index = False)

data_df_no_none = data_df_no_tags[~(data_df_no_tags["initial_response"]=="none")]
data_df_no_none = data_df_no_none[~(data_df_no_none["initial_response"]=="None")]
data_df_no_none

# Filtering out questions that contain word 'code'

In [ ]:
# Filtering out questions that contain word 'code'
code_df = data_df_no_none[data_df_no_none["input"].str.contains("code")]
display(code_df)
# code_df.to_csv('./data/final/code.csv', index = False)

data_df_no_code = data_df_no_none[~(data_df_no_none["input"].str.contains("code"))]
# data_df_no_code = data_df_no_tags
data_df_no_code

# Filtering out questions that contain word 'program'

In [ ]:
# Filtering out questions that contain word 'program'
program_df = data_df_no_code[data_df_no_code["input"].str.contains("program")]
display(program_df)
# program_df.to_csv('./data/final/program.csv', index = False)

data_df_no_program = data_df_no_code[~(data_df_no_code["input"].str.contains("program"))]
# data_df_no_program = data_df_no_code
data_df_no_program

# Filtering out questions that contain word 'compute'

In [ ]:
# Filtering out questions that contain word 'compute'
compute_df = data_df_no_program[data_df_no_program["input"].str.contains("compute ")]
display(compute_df)
# compute_df.to_csv('./data/final/compute.csv', index = False)

data_df_no_compute = data_df_no_program[~(data_df_no_program["input"].str.contains("compute "))]
# data_df_no_compute = data_df_no_program
data_df_no_compute

# Filtering out non-english responses

In [ ]:
# Filtering out non-english responses
'''
isEnglish function from 
https://stackoverflow.com/questions/27084617/detect-strings-with-non-english-characters-in-python
'''
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

not_english_df = data_df_no_compute[~(data_df_no_compute["initial_response"].apply(isEnglish))]
display(not_english_df)
# not_english_df.to_csv('./data/final/not_english.csv', index = False)

data_df_no_nonenglish = data_df_no_compute[data_df_no_compute["initial_response"].apply(isEnglish)]
data_df_no_nonenglish

# data_df_no_nonenglish = data_df_no_compute
data_df_no_nonenglish

# Filtering out rows with 'Response is incorrect' corrector thoughts

In [ ]:
# Filtering out rows with 'Response is incorrect' corrector thoughts
incorrect_response_df = data_df_no_nonenglish[data_df_no_nonenglish["response_problems"] == "The response is incorrect."]
display(incorrect_response_df)
# incorrect_response_df.to_csv('./data/final/incorrect_response.csv', index = False)

data_df_no_incorrect_response = data_df_no_nonenglish[~(data_df_no_nonenglish["response_problems"] == "The response is incorrect.")]
# data_df_no_incorrect_response = data_df_no_nonenglish
data_df_no_incorrect_response

# Filtering out questions that contain word 'formula'

In [ ]:
# Filtering out questions that contain word 'formula'
formula_df = data_df_no_incorrect_response[data_df_no_incorrect_response["input"].str.contains("formula")]
display(formula_df)
# formula_df.to_csv('./data/final/formula.csv', index = False)

final_df = data_df_no_incorrect_response[~(data_df_no_incorrect_response["input"].str.contains("formula"))]

# final_df = data_df_no_incorrect_response
final_df

# Replace the stopping sequence "Input:" that shows up in some of the responses with empty string and remove unwanted white spaces and new lines in responses.

In [ ]:
# Replace the stopping sequence "Input:" that shows up in some of the responses with empty string
final_df['initial_response'] = final_df['initial_response'].str.replace('Input:','')
final_df['corrected_reponse'] = final_df['corrected_reponse'].str.replace('Input:','')


# Remove unwanted white spaces and new lines in responses
final_df['initial_response'] = final_df['initial_response'].str.strip()
final_df['corrected_reponse'] = final_df['corrected_reponse'].str.strip()


final_df

# Save data to a csv file

In [ ]:
# final_df.to_csv("./data_test_synthetic_falcon/combined/initial_response_falcon_noprompt.csv", index=False)
final_df.to_csv("./data/final/helpful_aligner_data_new.csv", index=False)



# A word cloud visualization to show the most used words in generated responses. The most used words are in large font and the least used words in small font.

In [ ]:
# '''A word cloud visualization to show the most used words in generated responses. 
#    The most used words are in large font and the least used words in small font.
# '''
# text1 = " ".join(resp1 for resp1 in final_df.initial_response)
# text2 = " ".join(resp2 for resp2 in final_df.corrected_reponse)
# text = text1 +" "+ text2
# stopwords = set(STOPWORDS)
# wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)
# plt.figure( figsize=(30,20))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()

# Computing similarity between y and y'

In [ ]:
'''Similarity computation of y and y'
'''
model = SentenceTransformer('all-MiniLM-L6-v2')
similarity_scores = []

for i in range(len(final_df)):
    sentences = [final_df.iloc[i]['initial_response'], final_df.iloc[i]['corrected_reponse']]
    vectorized_sentences = model.encode(sentences)
    similarity_score = cos_sim(vectorized_sentences[0],vectorized_sentences[1]).item()
    similarity_scores.append(similarity_score)
#     if (i%100)==0:
#         print("Index: ", i, "Similarity score: ", similarity_score)

final_df['similarity_score'] = similarity_scores
final_df

In [ ]:
final_df.to_csv("./data/final/helpful_aligner_data_sim_scores.csv", index=False)

# Histogram of similarity scores between y and y'

In [ ]:
# Histogram of similarity scores between y and y'
sns.set(rc={'figure.figsize':(12,9)})
sns.histplot(data=final_df, x="similarity_score")

# Prepare Data for Inspector Training

In [ ]:
sim_data = pd.read_csv("./data/final/helpful_aligner_data_sim_scores.csv")
sim_data

In [ ]:
sim_data = sim_data[~(sim_data.similarity_score>=0.9)]
sim_data

# Reduce the sample size

In [ ]:
sim_data = sim_data[:70000]
sim_data

# Restructuring the data to have class 0 and class 1 labels for training the classification model

In [ ]:
INSTRUCTION_SEPARATOR = " [SEP] "
class0 = sim_data["input"] + INSTRUCTION_SEPARATOR + sim_data["initial_response"]
class1 = sim_data["input"] + INSTRUCTION_SEPARATOR + sim_data["corrected_reponse"]
df0 = pd.DataFrame(class0, columns = ["text"])
df0['label'] = 0
# df0.to_csv("./data/final/helpful_class0.csv", index=False)
df1= pd.DataFrame(class1, columns = ["text"])
df1['label'] = 1
# df1.to_csv("./data/final/helpful_class1.csv", index=False)

data = pd.concat([df0, df1]).reset_index(drop=True)
# data.to_csv("./data-inspector/final/helpful_inspector_data.csv", index=False)
data.to_csv("./data/final/helpful_inspector_data.csv", index=False)
data